# Reinforcement Learning GAN
Employ REINFORCE for training the Generator in a GAN formuluation.  As before, the reward to the Generator $G$ is simply the probability of real at each time point, $p_t$, assigned by the Discriminator $D$.

Classic GAN minimization objective,

$$\text{max}_{G_{\theta}} \text{min}_{D_{\theta}} \left[ \text{log}(D(x)) + \text{log}(1. - D(G(z)) \right]$$

In [147]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.distributions import Categorical

rnn_gen_size = 16
rnn_dis_size = 16
vocab_dim = 3
time_steps = 6
batch_size = 1
num_iterations = 500
gen_iterations = 1
dis_iterations = 1
print_every = 100
lr = 0.01
decay = 0.99

# TODO: Fix batching errors when initial model working.
assert batch_size == 1

## Generator and Discriminator.
Advantage function without a baseline may perform better for this task.  The hypothesis is that the pace of $D$'s learning can outpace the $G$'s learning.  So for instance, if the predictions from $D$ at a given time step, which we equate to reward $r_t$, decreases rapidly to 0, then the exponential moving average baseline will renormalize and leave little residual reward for a signal.

In [143]:
def score_function_with_mean_baseline(dt, val, rewards, decay = 0.99):
    '''Create a specific score function to pass to DistributionTensors'''
    reward = tf.reduce_mean(tf.add_n(rewards))
    ema = tf.train.ExponentialMovingAverage(decay)
    update_op = ema.apply([reward])
    baseline = ema.average(reward)
#     advantage = tf.stop_gradient(reward - baseline)
    advantage = tf.stop_gradient(reward)
    with tf.control_dependencies([update_op]):
        return dt.distribution.log_prob(val) * advantage


def generator(reuse=False):
    '''Define the Generator graph.'''
    # TODO: Generalize for random input.
    # batch_indices = gen_z
    batch_indices = tf.constant([[1., 0., 0.]], dtype=tf.float32)   
    
    with tf.variable_scope('gen', reuse=reuse):
        cell_gen = tf.nn.rnn_cell.BasicRNNCell(rnn_gen_size)
        state_gen = cell_gen.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn') as vs:
            sequence = []

            for t in xrange(time_steps):        
                if t > 0:
                    tf.get_variable_scope().reuse_variables()
                    
                rnn_in = batch_indices
                rnn_out, state_gen = cell_gen(rnn_in, state_gen)
                
                log_probs = tf.nn.log_softmax(
                    tf.contrib.layers.linear(rnn_out, vocab_dim, scope=vs))

                index = tf.contrib.bayesflow.stochastic_graph.DistributionTensor(
                    tf.contrib.distributions.Categorical,
                    logits=log_probs,
                    loss_fn=score_function_with_mean_baseline)
                batch_indices = tf.one_hot(index, vocab_dim, dtype=tf.float32)
                sequence.append(batch_indices)
    return tf.pack(sequence, axis=1)


def discriminator(sequence, reuse=False):
    '''Define the Discriminator graph.'''
    sequence = tf.unpack(sequence, axis=1)
    with tf.variable_scope('dis', reuse=reuse):    
        cell_dis = tf.nn.rnn_cell.BasicRNNCell(rnn_dis_size)
        state_dis = cell_dis.zero_state(batch_size, tf.float32)

        with tf.variable_scope('rnn') as vs:
            predictions = []

            for t, inp in enumerate(sequence):
                rnn_in = inp
                if t > 0:
                    tf.get_variable_scope().reuse_variables()

                rnn_out, state_dis = cell_dis(rnn_in, state_dis)
                pred = tf.nn.sigmoid(tf.contrib.layers.linear(rnn_out, 1, scope=vs))
                predictions.append(pred)
    return tf.pack(predictions, axis=1)

## Losses for Models.
The reward, which is the taken from the probability of real at each time step, is rescaled to a positive and negative range.

In [141]:
def generator_rewards(predictions):
    '''Generator rewards based on Discriminator predictions.'''
    predictions = tf.unpack(tf.squeeze(predictions, [2]), axis=1)
    
    # Modify predictions to have a larger dynamic range.
    predictions = [(p - 0.5) * 5 for p in predictions]
    final_reward = tf.contrib.bayesflow.stochastic_graph.surrogate_loss(predictions)
    return final_reward
    
        
def discriminator_loss(predictions, labels):
    '''Discriminator loss based on predictions and labels'''
    loss = tf.contrib.losses.log_loss(tf.squeeze(predictions, [2]), labels)
    final_loss = tf.reduce_sum(loss) / batch_size / time_steps
    return final_loss

## Generative Adversarial Networks Training.

In [148]:
tf.reset_default_graph()

# Training data.
real_sequence = tf.constant([[[1.,0.,0.],
                              [0.,1.,0.],
                              [0.,0.,1.],
                              [1.,0.,0.],
                              [0.,1.,0.],
                              [0.,0.,1.]]], dtype=tf.float32)

real_labels = tf.ones((batch_size, time_steps))
fake_labels = tf.zeros((batch_size, time_steps))

# Generator loss.
fake_sequence = generator()
fake_predictions = discriminator(fake_sequence)
gen_reward = generator_rewards(fake_predictions)

# Discriminator loss.
real_predictions = discriminator(real_sequence, reuse=True)
dis_loss_real = discriminator_loss(real_predictions, real_labels)
dis_loss_fake = discriminator_loss(fake_predictions, fake_labels)
dis_loss = dis_loss_real + dis_loss_fake


with tf.name_scope('train_generator'):
    gen_optimizer = tf.train.AdamOptimizer(lr)
    gen_vars = [v for v in tf.trainable_variables() if v.op.name.startswith('gen')]
    gen_train_op = gen_optimizer.minimize(-gen_reward, var_list = gen_vars)

with tf.name_scope('train_discriminator'):
    dis_optimizer = tf.train.AdamOptimizer(lr)
    dis_vars = [v for v in tf.trainable_variables() if v.op.name.startswith('dis')]
    dis_train_op = dis_optimizer.minimize(dis_loss, var_list = dis_vars)


with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    print 'num_iterations', num_iterations
    for t in xrange(num_iterations):
        for i in xrange(dis_iterations):
            _, dis_loss_eval = sess.run([dis_train_op, dis_loss])

        for i in xrange(gen_iterations):
            _, gen_loss_eval = sess.run([gen_train_op, gen_reward])

        if t % print_every == 0:
            print 'Dis loss:', dis_loss_eval
            print 'Gen loss:', gen_loss_eval[0]
                      
    print 'fake:', sess.run([fake_sequence, fake_predictions]), '\n'
    print 'real:', sess.run([real_sequence, real_predictions]), '\n'
#     print sess.run(dis_vars)



num_iterations 500
Dis loss: 0.25212
Gen loss: -8.05365
Dis loss: 0.091378
Gen loss: 0.15149
Dis loss: 0.166207
Gen loss: -4.38635
Dis loss: 0.275505
Gen loss: -2.72163
Dis loss: 0.231751
Gen loss: 0.140702
fake: [array([[[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]]], dtype=float32), array([[[ 0.49679664],
        [ 0.50220495],
        [ 0.50097305],
        [ 0.5017724 ],
        [ 0.50207412],
        [ 0.50715679]]], dtype=float32)] 

real: [array([[[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]]], dtype=float32), array([[[ 0.49679664],
        [ 0.50220495],
        [ 0.50097305],
        [ 0.5017724 ],
        [ 0.50207412],
        [ 0.50715679]]], dtype=float32)] 



# TODO
* Generalize for random input $z_{gen}$
* Fix batching

### Check Loss Calculation

In [ ]:
logits = tf.constant([[[0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5]]], dtype=tf.float32)
labels = tf.constant([[0, 0, 0, 0], [0, 0, 0, 0]])

loss = tf.contrib.losses.log_loss(logits, labels)
with tf.Session() as sess:
    print sess.run(loss)


## Real Data.
Create a simple data distribution with sequential structure for testing the GAN.

In [ ]:
num_examples = 1000

# Real Sequence (num_examples, time_steps, inp_dim)
real_sequences = []
real_seq = np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.],[1.,1.,1.]])

for i in xrange(num_examples):
    real_sequences.append(real_seq)
real_sequences = np.asarray(real_sequences)

# Real Labels (num_examples, time_steps)
real_labels = np.ones([num_examples, time_steps], dtype=np.int)